# Vérifier et préparer votre environnement de projet

## Vérifier votre environnement

In [ ]:
!gcloud config get-value core/account

In [ ]:
!gcloud config get-value core/project

## Vérifier que les APIs sont activées

In [ ]:
!gcloud services list --enabled --filter speech

## Vérifier que les API Speech-to-Text et Text-to-Speech sont bien activées

In [ ]:
!gcloud services list --enabled --filter=speech

## Vérifier que l'API Translation est bien activée

In [ ]:
!gcloud services list --enabled --filter=translate

## Activer les APIs éventuellement non activées
Si certaines APIs ne sont pas encore activées, retournez sur l'interface web de la console GCP, cliquez sur __Navigation Menu__ (icône dite "Burger menu") > __APIs & Services__ > __Library__.

Cherchez la ou les API(s) non encore activées et cliquez sur __Enable__.

## Déclarer votre clé d'API
Retrouvez la valuer de votre clé d'API :
- depuis l'interface graphique de la console GCP :
cliquez sur __Navigation Menu__ (icône dite "Burger menu") > __APIs & Services__ > __Credentials__

- depuis le fichier dans lequel vous l'avez enregistrée :
Dans __Cloud Shell__ tapez la commende :

```cat My-API-key.txt```

Puis copiez-collez la valeur ci-dessous :

In [ ]:
%env API_KEY <valeur>

In [ ]:
PROJECT = !gcloud config get-value core/project
%env GOOGLE_CLOUD_PROJECT {PROJECT[0]}

# Transcrire un fichier audio vocal vers du texte avec Speech-to-Text API

## Créer un service account et télécharger la clé

In [ ]:
!gcloud iam service-accounts create my-lab-sa --display-name "my lab service account"

In [ ]:
!gcloud iam service-accounts keys create ~/key.json --iam-account my-lab-sa@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS="/home/jupyter/key.json"

## Utiliser la librairie client Python 

Vous allez lancer la transcription d'un [autre fichier sonore vocal](https://storage.googleapis.com/speech-language-samples/fr-sample.flac) (cliquez sur le lien pour l'écouter).

### Vérifier d'abord que le module google-cloud-speech est bien installé :

In [ ]:
!pip freeze | grep speech

### Ensuite, préparer l'appel à l'API avec le client Python :

In [ ]:
from google.cloud import speech

client = speech.SpeechClient.from_service_account_json('/home/jupyter/key.json')

gcs_uri = "gs://speech-language-samples/fr-sample.flac"

audio = speech.types.RecognitionAudio(uri=gcs_uri)
config = speech.types.RecognitionConfig(
    encoding=speech.types.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code="fr",
)

### Lancer l'opération en mode synchrone et afficher la meilleure réponse :

In [ ]:
response = client.recognize(config=config, audio=audio)
for result in response.results:
    # The first alternative is the most likely one for this portion.
    print(u"Transcript: {}".format(result.alternatives[0].transcript))
    print("Confidence: {}".format(result.alternatives[0].confidence))

### Transcrire un autre enregistrement vocal

Vous allez lancer la transcription d'un [nouveau fichier sonore vocal](https://storage.googleapis.com/ml-api-codelab/tr-ostrich.wav) (cliquez sur le lien pour l'écouter).

Adaptez le code ci-dessous afin de préparer la transcription de ce nouvel enregistrement (indice : le fichier est un .WAV en Turc) :

In [ ]:
gcs_uri = "gs://speech-language-samples/fr-sample.flac"

audio = speech.types.RecognitionAudio(uri=gcs_uri)
config = speech.types.RecognitionConfig(
    encoding=speech.types.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code="fr",
)

### Lancer la transcription en mode asynchrone et afficher la meilleure réponse :

In [ ]:
operation = client.long_running_recognize(
    config=config, audio=audio
)
print("Waiting for operation to complete...")
response = operation.result(timeout=90)

In [ ]:
for result in response.results:
    # The first alternative is the most likely one for this portion.
    print(u"Transcript: {}".format(result.alternatives[0].transcript))
    print("Confidence: {}".format(result.alternatives[0].confidence))

# Traduire du texte avec Translation API

## Stocker le texte précédent dans une variable

In [ ]:
text = response.results[0].alternatives[0].transcript

## Initialiser le client de l'API Translation

In [ ]:
from google.cloud import translate_v2 as translate

translate_client = translate.Client.from_service_account_json('/home/jupyter/key.json')

## Préparer la requête / spécifier la langue cible

In [ ]:
"""Translates text into the target language.

Target must be an ISO 639-1 language code.
See https://cloud.google.com/translate/docs/reference/rest/v3/SupportedLanguages
"""
target = "fr"

## Exécuter l'appel API

In [ ]:
# Text can also be a sequence of strings, in which case this method
# will return a sequence of results for each text.
result = translate_client.translate(text, target_language=target)

## Afficher le résultat

In [ ]:
print(u"Text: {}".format(result["input"]))
print(u"Translation: {}".format(result["translatedText"]))
print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

# Synthèse vocale avec Text-to-Speec API

## Préparer l'environnement Python

In [ ]:
!pip install --upgrade google-cloud-texttospeech

## Initialiser le client Python

In [ ]:
from google.cloud import texttospeech

tts_client = texttospeech.TextToSpeechClient.from_service_account_json('/home/jupyter/key.json')

## Imprimer la liste des voix disponibles

In [ ]:
voices = tts_client.list_voices()

for voice in voices.voices:
    # Display the voice's name. Example: tpc-vocoded
    print(f"Name: {voice.name}")

    # Display the supported language codes for this voice. Example: "en-US"
    for language_code in voice.language_codes:
        print(f"Supported language: {language_code}")

    ssml_gender = texttospeech.SsmlVoiceGender(voice.ssml_gender)

    # Display the SSML Voice Gender
    print(f"SSML Voice Gender: {ssml_gender.name}")

    # Display the natural sample rate hertz for this voice. Example: 24000
    print(f"Natural Sample Rate Hertz: {voice.natural_sample_rate_hertz}\n")

## Stocker la requete dans un dictionnaire Python

In [ ]:
my_text = 'Cloud Text-to-Speech API allows developers to include natural-sounding, synthetic human speech as playable audio in their applications. The Text-to-Speech API converts text or Speech Synthesis Markup Language (SSML) input into audio data like MP3 or LINEAR16 (the encoding used in WAV files).'

## Préparer, lancer et exécuter la requête

In [ ]:
input_text = texttospeech.SynthesisInput(text=my_text)

voice = texttospeech.VoiceSelectionParams(
    language_code="en-GB",
    name="en-GB-Standard-A",
    ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

response = tts_client.synthesize_speech(
    request={"input": input_text, "voice": voice, "audio_config": audio_config}
)

## Enregistrer la réponse dans un fichier binaire / MP3

In [ ]:
with open("output.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

In [ ]:
!ls

## Stocker ce fichier dans Google Cloud Storage

In [ ]:
username = !gcloud config get-value core/account
bucketname = username[0] + "-bucket"
!gsutil  mb -b on -l us-east1 gs://{bucketname}/
!gsutil iam ch allUsers:objectViewer gs://{bucketname}/

## Copier le fichier dans GCS

In [ ]:
!gsutil cp output.mp3 gs://{bucketname}/

## Vous pouvez maintenant écouter ce fichier en copiant le lien ci-dessous.

In [ ]:
print("https://storage.googleapis.com/" + bucketname + "/output.mp3")